In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import IsolationForest
from sklearn.metrics import classification_report, accuracy_score
import numpy as np
import xgboost as xgb

In [2]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix

In [3]:
urls = []
comments = []

# urls.append("https://raw.githubusercontent.com/sudomaster00081/MscProj/main/Implimentation/som_data.csv")
comments.append("self made data")

urls.append("https://raw.githubusercontent.com/sudomaster00081/MscProj/main/Implimentation/DATASET/caida2007/csv/2ssplit%20combined%20dataset%20CAIDA%202007%20filled.csv")



url = urls[0]


# Step 1: Load and Prepare Data

# Load data from CSV
data = pd.read_csv(url)
df = data

In [4]:
### PRE

In [5]:
# Define the function for normalization using tanh-estimator method, excluding specified columns
def normalize_with_tanh_estimator(data, mean_std_dict, exclude_cols):
    normalized_data = pd.DataFrame()
    for col in data.columns:
        if col not in exclude_cols:
            mu, sigma = mean_std_dict[col]
            normalized_col = 0.5 * (np.tanh(0.1 * ((data[col] - mu) / sigma)) + 1)
            normalized_data[col] = normalized_col
        else:
            normalized_data[col] = data[col]  # Keep excluded columns as they are
    return normalized_data

In [6]:

def true_vs_predicted(y_true, y_pred):

  # Compute confusion matrix
  cm = confusion_matrix(y_true, y_pred)

  # Extract counts from confusion matrix
  attack_as_normal = cm[1, 0]
  normal_as_attack = cm[0, 1]
  attack_as_attack = cm[1, 1]
  normal_as_normal = cm[0, 0]

  # Print the results
  print("attack (1) but predicted as normal =", attack_as_normal)
  print("normal (0) but predicted as attack =", normal_as_attack)
  print("attack (1) predicted as attack =", attack_as_attack)
  print("normal (0) predicted as normal =", normal_as_normal)


In [7]:
# Define the columns to exclude from normalization
exclude_cols = ['label']  # Add more columns here if needed
  # Calculate and store the mean and standard deviation for each dimension, excluding specified columns
mean_std_dict = {}
for col in df.columns:
    if col not in exclude_cols:
        mean_std_dict[col] = (df[col].mean(), df[col].std())

  # Normalize the dataset using tanh-estimator method, excluding specified columns
normalized_df = normalize_with_tanh_estimator(df, mean_std_dict, exclude_cols)


In [8]:




# Separate features and target

X = data.drop(columns = ['label','bandwidth'], axis=1)  # Features
y = data['label']               # Target

# Step 2: Train-Test Split

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Step 3: Train the Isolation Forest Model



In [9]:

# Initialize Isolation Forest model
iso_forest = IsolationForest(random_state=42)

# Fit the model
iso_forest.fit(X_train)

# Step 4: Predict and Evaluate

# Predict anomaly scores
train_anomaly_scores = iso_forest.decision_function(X_train)
test_anomaly_scores = iso_forest.decision_function(X_test)

# Set threshold to classify as normal (0) or anomaly (1)
threshold = 0

# Convert anomaly scores to binary labels
y_pred_train = (train_anomaly_scores >= threshold).astype(int)
y_pred_test = (test_anomaly_scores >= threshold).astype(int)

# Evaluate performance
print("Training Accuracy:", accuracy_score(y_train, y_pred_train))
print("Testing Accuracy:", accuracy_score(y_test, y_pred_test))

print("\nClassification Report (Testing Set):\n", classification_report(y_test, y_pred_test))
true_vs_predicted(y_test, y_pred_test)


Training Accuracy: 0.7193119624706802
Testing Accuracy: 0.726775956284153

Classification Report (Testing Set):
               precision    recall  f1-score   support

           0       0.63      0.43      0.51       182
           1       0.76      0.87      0.81       367

    accuracy                           0.73       549
   macro avg       0.69      0.65      0.66       549
weighted avg       0.71      0.73      0.71       549

attack (1) but predicted as normal = 46
normal (0) but predicted as attack = 104
attack (1) predicted as attack = 321
normal (0) predicted as normal = 78


In [10]:
## for xg boost testing

# from sklearn.model_selection import GridSearchCV

# # Define a grid of hyperparameters to search over
# param_grid = {
#     'learning_rate': [0.1, 0.2, 0.3],
#     'max_depth': [3, 5, 7],
#     'n_estimators': [50, 100, 200],
#     'gamma': [0, 0.1, 0.2]
# }

# # Initialize XGBoost model
# xgb_model = xgb.XGBClassifier(random_state=42)

# # Grid search cross-validation
# grid_search = GridSearchCV(estimator=xgb_model, param_grid=param_grid, cv=3, scoring='accuracy', verbose=1)
# grid_search.fit(X_train, y_train)

# # Print best parameters and best score
# print("Best Parameters:", grid_search.best_params_)
# print("Best Cross-validation Accuracy:", grid_search.best_score_)

# # Use the best model found by grid search
# best_xgb_model = grid_search.best_estimator_

# # Fit and evaluate the best model
# best_xgb_model.fit(X_train, y_train)
# y_pred_test_best = best_xgb_model.predict(X_test)
# print("Best Model Testing Accuracy:", accuracy_score(y_test, y_pred_test_best))


In [11]:
### Random forest test data

# from sklearn.model_selection import GridSearchCV

# # Define a grid of hyperparameters to search over
# param_grid = {
#     'n_estimators': [50, 100, 200],
#     'max_depth': [None, 10, 20, 30],
#     'min_samples_split': [2, 5, 10],
#     'min_samples_leaf': [1, 2, 4],
#     'max_features': ['auto', 'sqrt', 'log2']
# }

# # Initialize Random Forest model
# rf_model = RandomForestClassifier(random_state=42)

# # Grid search cross-validation
# grid_search = GridSearchCV(estimator=rf_model, param_grid=param_grid, cv=3, scoring='accuracy', verbose=1)
# grid_search.fit(X_train, y_train)

# # Print best parameters and best score
# print("Best Parameters:", grid_search.best_params_)
# print("Best Cross-validation Accuracy:", grid_search.best_score_)

# # Use the best model found by grid search
# best_rf_model = grid_search.best_estimator_

# # Fit and evaluate the best model
# best_rf_model.fit(X_train, y_train)
# y_pred_test_best = best_rf_model.predict(X_test)
# print("Best Model Testing Accuracy:", accuracy_score(y_test, y_pred_test_best))


In [12]:
from sklearn.ensemble import RandomForestClassifier

# Initialize Random Forest model
rf_model = RandomForestClassifier(random_state=42)

# Fit the model
rf_model.fit(X_train, y_train)

# Predict on training and testing sets
y_pred_train_rf = rf_model.predict(X_train)
y_pred_test_rf = rf_model.predict(X_test)

# Evaluate performance
print("Random Forest Training Accuracy:", accuracy_score(y_train, y_pred_train_rf))
print("Random Forest Testing Accuracy:", accuracy_score(y_test, y_pred_test_rf))

print("\nRandom Forest Classification Report (Testing Set):\n", classification_report(y_test, y_pred_test_rf))
true_vs_predicted(y_test, y_pred_test_rf)

Random Forest Training Accuracy: 1.0
Random Forest Testing Accuracy: 0.9836065573770492

Random Forest Classification Report (Testing Set):
               precision    recall  f1-score   support

           0       0.95      1.00      0.98       182
           1       1.00      0.98      0.99       367

    accuracy                           0.98       549
   macro avg       0.98      0.99      0.98       549
weighted avg       0.98      0.98      0.98       549

attack (1) but predicted as normal = 9
normal (0) but predicted as attack = 0
attack (1) predicted as attack = 358
normal (0) predicted as normal = 182


In [13]:
import xgboost as xgb
from sklearn.metrics import accuracy_score, classification_report

# Initialize XGBoost model
xgb_model = xgb.XGBClassifier(random_state=42)

# Fit the model
xgb_model.fit(X_train, y_train)

# Predict on training and testing sets
y_pred_train_xgb = xgb_model.predict(X_train)
y_pred_test_xgb = xgb_model.predict(X_test)

# Evaluate performance
print("XGBoost Training Accuracy:", accuracy_score(y_train, y_pred_train_xgb))
print("XGBoost Testing Accuracy:", accuracy_score(y_test, y_pred_test_xgb))

print("\nXGBoost Classification Report (Testing Set):\n", classification_report(y_test, y_pred_test_xgb))
true_vs_predicted(y_test, y_pred_test_xgb)


XGBoost Training Accuracy: 0.9992181391712275
XGBoost Testing Accuracy: 0.9781420765027322

XGBoost Classification Report (Testing Set):
               precision    recall  f1-score   support

           0       0.95      0.99      0.97       182
           1       0.99      0.97      0.98       367

    accuracy                           0.98       549
   macro avg       0.97      0.98      0.98       549
weighted avg       0.98      0.98      0.98       549

attack (1) but predicted as normal = 10
normal (0) but predicted as attack = 2
attack (1) predicted as attack = 357
normal (0) predicted as normal = 180


In [14]:
pd_benign = pd.read_csv("https://raw.githubusercontent.com/sudomaster00081/MscProj/main/Implimentation/DATASET/isd%202017/ids-2017-benign-Monday-WorkingHours2s%20unlabeled_unfilled_entropy.csv")

In [15]:
pd_benign

,etpSrcIP,etpSrcP,etpDstP,etpProtocol,totalPacket,bandwidth,label
0,-0.000000,-0.000000,-0.000000,-0.000000,8.0,480.0,0
1,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0
2,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0
3,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0
4,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0
...,...,...,...,...,...,...,...
14563,3.250311,3.885461,3.771368,0.958712,84.0,5817.0,0
14564,3.628511,4.031906,3.741210,0.670525,74.0,5046.0,0
14565,3.959248,4.166415,4.471768,-0.000000,124.0,8618.0,0
14566,3.343581,3.885510,3.721702,0.238685,51.0,3440.0,0


In [16]:
X_benign = pd_benign.drop(columns = ['label','bandwidth'], axis=1)  # Features
y_benign = pd_benign['label']

In [17]:
y_benign.sum()

0

In [18]:
y_benign

0        0
1        0
2        0
3        0
4        0
        ..
14563    0
14564    0
14565    0
14566    0
14567    0
Name: label, Length: 14568, dtype: int64

In [19]:
benign_rf_pred = rf_model.predict(X_benign)
benign_xgb_pred = xgb_model.predict(X_benign)
# Evaluate performance
print("Random Forest Testing Accuracy:", accuracy_score(y_benign, benign_rf_pred))
print("XGBoost Testing Accuracy:", accuracy_score(y_benign, benign_xgb_pred))

print("\nRandom Forest Classification Report (Testing Set):\n", classification_report(y_benign, benign_rf_pred,zero_division=1))
true_vs_predicted(y_benign, benign_rf_pred)

print("\nXGBoost Classification Report (Testing Set):\n", classification_report(y_benign, benign_xgb_pred,zero_division=1))
true_vs_predicted(y_benign, benign_xgb_pred)


Random Forest Testing Accuracy: 0.9840060406370126
XGBoost Testing Accuracy: 0.9250411861614497

Random Forest Classification Report (Testing Set):
               precision    recall  f1-score   support

           0       1.00      0.98      0.99     14568
           1       0.00      1.00      0.00         0

    accuracy                           0.98     14568
   macro avg       0.50      0.99      0.50     14568
weighted avg       1.00      0.98      0.99     14568

attack (1) but predicted as normal = 0
normal (0) but predicted as attack = 233
attack (1) predicted as attack = 0
normal (0) predicted as normal = 14335

XGBoost Classification Report (Testing Set):
               precision    recall  f1-score   support

           0       1.00      0.93      0.96     14568
           1       0.00      1.00      0.00         0

    accuracy                           0.93     14568
   macro avg       0.50      0.96      0.48     14568
weighted avg       1.00      0.93      0.96     145

In [20]:
benign_rf_pred.sum()

233

In [21]:
benign_xgb_pred.sum()

1092

In [22]:
len(y_benign)

14568

In [23]:
y_benign.sum()

0

# Test here

In [24]:
pd_test0 = pd.read_csv("https://raw.githubusercontent.com/sudomaster00081/MscProj/main/Implimentation/DATASET/caida2007/csv/2ssplit%20combined%20dataset%20CAIDA%202007%20filled.csv")

In [25]:
pd_test0

,etpSrcIP,etpSrcP,etpDstP,etpProtocol,totalPacket,bandwidth,label
0,2.519621,-0.000000,-0.000000,-0.000000,700.0,19600.0,0
1,2.520031,-0.000000,-0.000000,-0.000000,667.0,18676.0,0
2,2.542279,0.169128,0.115456,0.115456,709.0,20072.0,0
3,2.534989,0.015023,0.015023,0.015023,729.0,20432.0,0
4,2.447196,0.858468,0.394699,0.394699,796.0,23528.0,0
...,...,...,...,...,...,...,...
1823,11.923836,1.370867,0.882627,0.370512,337886.0,9941004.0,1
1824,11.922541,1.338163,0.861861,0.363928,334105.0,9817640.0,1
1825,11.920152,1.314635,0.845645,0.359186,341650.0,10031952.0,1
1826,11.924613,1.276776,0.823399,0.351973,335898.0,9850308.0,1


In [26]:
X_benign = pd_test0.drop(columns = ['label','bandwidth'], axis=1)  # Features
y_benign = pd_test0['label']

In [27]:
benign_rf_pred = rf_model.predict(X_benign)
# Evaluate performance
print("Random Forest Testing Accuracy:", accuracy_score(y_benign, benign_rf_pred))

print("\nRandom Forest Classification Report (Testing Set):\n", classification_report(y_benign, benign_rf_pred))

Random Forest Testing Accuracy: 0.9950765864332604

Random Forest Classification Report (Testing Set):
               precision    recall  f1-score   support

           0       0.99      1.00      0.99       600
           1       1.00      0.99      1.00      1228

    accuracy                           1.00      1828
   macro avg       0.99      1.00      0.99      1828
weighted avg       1.00      1.00      1.00      1828

